In [6]:
# relevant packages & modules
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import requests
import urllib.request
import json
from tqdm import tqdm
# from tqdm.notebook import tqdm
import undetected_chromedriver.v2 as uc
from IPython.display import clear_output
import pandas as pd


# Have powerfull bot protection, so we need undetected_chrome
class ScrapperPlusvalia:
    
    driver = None

    
    logged_in = False
    
    def __init__(self):
        
        self.driver = uc.Chrome(version_main=95)
        self.root = "https://www.plusvalia.com/inmuebles-ordenado-por-fechaonline-descendente"
        self.base = f"{self.root}-pagina-"
        self.driver.get(f"{self.base}1.html")

    def wait(self, t):
        time.sleep(t)
        
    def check_requisites(self):     
        return True
    
    def xp(self,xpath):
        return self.driver.find_elements(By.XPATH, xpath)

    def collect_urls(self,page_from,page_to,waiting_time=2)->list:
        
        if not self.check_requisites():
            return []

        urls = []
        repeated = []
        

        with open("urls.txt","a") as f:

            for page_i in tqdm(range(page_from, page_to+1)):

                url = f"{self.base}{page_i}.html"
                self.driver.get(url)
                
                chunk = self.driver.find_elements(By.XPATH, '//div[contains(@class,"list-card-container")]/div')
                
                for c in chunk:
                    if str(c.get_attribute("class")) != "ads-container":

                        url_item = str(c.get_attribute("data-to-posting"))
                        if url_item is None:
                            pass
                        else:
                            if url_item not in urls:
                                urls.append(url_item)
                            else:
                                repeated.append(f"Repeated inmuebles-ordenado-por-fechaonline-descendente-{page_i}.html:{url_item}")

                if len(urls) > 0:
                    f.write("\n".join(map(lambda s: f"https://www.plusvalia.com{s}", urls)))
                    urls = []

                current_url = self.driver.current_url


                if current_url == f"{self.root}.html":
                    page = 1
                else:
                    page = int(re.match(r"^https\:\/\/www\.plusvalia\.com\/[A-z\-]*(\d+)\.html$", current_url).groups()[0])

                if page == page_to:
                    break
                
                # next = self.driver.find_elements(By.CSS_SELECTOR, 'li.pag-go-next > a')[0]
                # next.click()
                self.wait(waiting_time)

        return urls, repeated


    def scrape_items(self,urls_items)->list:
        
        if not self.check_requisites():
            return []

        
    
    def close(self):
        if self.driver:
            self.driver.close()
            
    def __del__(self):
        print("Closing...")
        self.close()

In [137]:
scrapper = ScrapperPlusvalia()
# urls, _ = scrapper.collect_urls(1,3468,1.5)  # 3468

# Get urls of items

In [92]:
import pandas as pd
import numpy as np
with open("urls.txt") as f:
    lines = list(filter(lambda s: "htmlhttps" not in s,f.readlines()))
    print(len(lines))
    lines = np.unique(lines)
    print(len(lines))

    with open("urls_clean.txt","w") as f_out:
        f_out.writelines(lines)

with open("urls_clean.txt","r") as f_in:
    urls = f_in.readlines()
    print(len(urls))

34850
16738
16737


# Extract items

In [140]:
class ItemRealState:

    def __init__(self) -> None:

        self.type_real_state="none"
        self.url="none"
        self.title="none"
        self.location_string="none"
        self.location_latitude="none"
        self.location_longitude="none"
        self.date_publication="none"
        self.state="none"
        self.deliver="none"
        self.coin="none"
        self.views="none"
        self.price=0
        self.expenses="none"
        self.discount=0
        self.description_sup="none"
        self.description="none"
        self.pictures="none"
        self.advertiser_name="none"
        self.advertiser_codes="none"
        self.feature_unities="none"
        self.feature_stotal="none"
        self.feature_scover="none"
        self.feature_bath="none"
        self.feature_parking="none"
        self.feature_toilete="none"
        self.feature_antique="none"
        self.feature_rooms="none"
        self.general_features_xml="none"
        self.location_tree="none"

    def extract_first(self, elements, mapping=None, default="none"):
        elements_len = len(elements)
        if elements_len == 0:
            return default        
        try:
            element = elements[0].text
            if mapping is not None:
                element = mapping(element)
            return element
        except:
            return default

    def extract_multiple(self, web_elements, separator="|"):
        
        out = ""
        for i in range(len(web_elements)):
            out += web_elements[i].text
            if i < len(web_elements)-1:
                out += separator        
        return out

    def get_serie(self, map_attributes_to_columns=dict()):
        d = self.getAttributesAsDict()
        if len(d) == 0:
            print("First add atributes with set_attributes")
            return pd.Series()

        if len(map_attributes_to_columns) > 0:
            # not supported
            return pd.Series()
        
        return pd.Series(data=d)


    def parse_attributes(self,scrapper):
        today = datetime.date.today()

        def dateToString(date):
            return f"{date.day}/{date.month}/{date.year}"
        today_str = dateToString(today)

        web_elements_location_tree = scrapper.driver.find_elements(By.XPATH, "//ul[@class='breadcrumb']/li")
        self.url = scrapper.driver.current_url
        self.location_tree = self.extract_multiple(web_elements_location_tree,"->")
        self.type_real_state = web_elements_location_tree[1].text.strip().lower()

        if self.type_real_state != "proyectos": # in ["casa", "oficina comercial","departamento","terreno / lote","bodega-galpón"]:
            self.title = self.extract_first(scrapper.xp("//hgroup[@class='title-container']//div[@class='section-title']"))
            self.location_string = self.extract_first(scrapper.xp("//h2[@class='title-location']")).replace("\nVer en mapa", "")
            url_maps = scrapper.driver.find_elements(By.XPATH, "//img[@id='static-map']")[0].get_attribute("src")

            raw_lat_lon = re.match(r".*center=(\-?\d+\.[\d]+),(\-?\d+\.[\d]+).*",url_maps)
            if raw_lat_lon:
                self.location_latitude, self.location_longitude = map(float,raw_lat_lon.groups())
            
            self.date_publication = self.extract_first(scrapper.driver.find_elements(By.XPATH, "//div[@id='user-views']/div/div[1]"))
            self.views = int(self.extract_first(scrapper.driver.find_elements(By.XPATH, "//div[@id='user-views']/div/div[2]"),default="0 visualizaciones").split(" ")[0])
            if self.date_publication == "Publicado hoy":
                self.date_publication = today_str
            else:
                amount = -1
                time_type = "unknown"
                try:
                    gr = re.match("Publicado [^\d]*(\d+) ([A-zñáéíóú]+)", self.date_publication).groups()
                        
                    amount = int(gr[0])
                    time_type = gr[1]

                except Exception as e:
                    pass

                if amount >= 0:
                    amount = int(amount)
                    if "día" in time_type.lower() or "dia" in time_type.lower():
                        self.date_publication = dateToString(today - datetime.timedelta(days=amount))
                    elif "año" in time_type:
                        self.date_publication = dateToString(today - datetime.timedelta(days=amount*365))
                    else:
                        self.date_publication = "unknown"
                        print(f"Unknown {self.date_publication}")
                else:
                    self.date_publication = "unknown"

            self.state = self.extract_first(scrapper.driver.find_elements(By.XPATH, "//div[@class='price-operation']"))
            self.deliver = "house_default"
            price_items_split = self.extract_first(scrapper.driver.find_elements(By.XPATH, "//div[@class='price-items']/span/span")).split("\n")
            self.coin, self.price = re.match("([A-z]+) ([\d\.]+)", price_items_split[0]).groups()
            self.price = float(self.price.replace(".",""))
            self.discount = 0 if len(price_items_split) == 1 else int(price_items_split[1].replace("%",""))
            
            self.expenses = self.extract_first(scrapper.xp("//div[contains(@class,'block-expensas')]"))

            

            self.description_sup = self.extract_first(scrapper.driver.find_elements(By.XPATH, "//h2[@class='title-type-sup']")).replace("\n","|")
            self.description = self.extract_first(scrapper.driver.find_elements(By.XPATH, "//div[@id='longDescription']")).replace("\n","|")
            
            web_elements_imgs = scrapper.driver.find_elements(By.XPATH, "//div[@id='react-gallery']//img")
            self.pictures = ""
            for i,element in enumerate(web_elements_imgs):
                self.pictures += element.get_attribute("src")
                if i < len(web_elements_imgs)-1:
                    self.pictures += "|"
            
            self.advertiser_name = self.extract_first(scrapper.driver.find_elements(By.XPATH, "//section[@id='reactPublisherData']/div/div")).replace("\n","|")
            self.advertiser_codes = self.extract_multiple(scrapper.driver.find_elements(By.XPATH, "//section[@id='reactPublisherData']/div/ul/li")).replace("\n","|")

            features = scrapper.xp("//ul[@class='section-icon-features']//li")
            for feature in features:
                text = feature.text
                if feature.find_elements(By.XPATH, ".//i[@class='icon-stotal']"):
                    self.feature_stotal = text
                elif feature.find_elements(By.XPATH, ".//i[@class='icon-scubierta']"):
                    self.feature_scover = text
                elif feature.find_elements(By.XPATH, ".//i[@class='icon-bano']"):
                    self.feature_bath = text
                elif feature.find_elements(By.XPATH, ".//i[@class='icon-cochera']"):
                    self.feature_parking = text
                elif feature.find_elements(By.XPATH, ".//i[@class='icon-toilete']"):
                    self.feature_toilete = text
                elif feature.find_elements(By.XPATH, ".//i[@class='icon-antiguedad']"):
                    self.feature_antique = text
                elif feature.find_elements(By.XPATH, ".//i[@class='icon-dormitorio']"):
                    self.feature_rooms = text
                else:
                    i = feature.find_elements(By.XPATH, ".//i")[0]
                    print(f"unknown {text}", i.get_attribute("class"), i.text)


            general_features = scrapper.xp("//div[@id='reactGeneralFeatures']//div[contains(@class,'AccordionItem')]")
            self.general_features_xml = "<general_features_xml>"
            for feature in general_features:
                type_title = feature.find_element(By.XPATH, ".//div[contains(@class,'AccordionHeader')]").text.strip()
                content = feature.find_element(By.XPATH, ".//div[contains(@class,'AccordionContent')]").text.strip().replace("\n","|")

                element = f"<general_feature><title>{type_title}</title><content>{content}</content></general_feature>"
                self.general_features_xml += element

            self.general_features_xml += "</general_features_xml>"


            # scrapper.wait(1)

        elif self.type_real_state == "proyectos":
            
            self.title = self.extract_first(scrapper.xp("//h2[@class='title']"))
            self.location_string = self.extract_first(scrapper.xp("//p[@class='subtitle']")).replace("\nVer en mapa", "")
            url_maps = scrapper.driver.find_elements(By.XPATH, "//img[@id='static-map']")[0].get_attribute("src")
            
            raw_lat_lon = re.match(r".*center=(\-?\d+\.[\d]+),(\-?\d+\.[\d]+).*",url_maps)
            if raw_lat_lon:
                self.location_latitude, self.location_longitude = map(float,raw_lat_lon.groups())

            self.date_publication = self.extract_first(scrapper.driver.find_elements(By.XPATH, "//div[@id='user-views']/div/div[1]"))
            self.views = int(self.extract_first(scrapper.driver.find_elements(By.XPATH, "//div[@id='user-views']/div/div[2]"),default="0 visualizaciones").split(" ")[0])
            if self.date_publication == "Publicado hoy":
                self.date_publication = today_str
            else:
                amount = -1
                time_type = "unknown"
                try:
                    gr = re.match("Publicado [^\d]*(\d+) ([A-zñáéíóú]+)", self.date_publication).groups()
                        
                    amount = int(gr[0])
                    time_type = gr[1]

                except Exception as e:
                    pass

                if amount >= 0:
                    amount = int(amount)
                    if "día" in time_type.lower() or "dia" in time_type.lower():
                        self.date_publication = dateToString(today - datetime.timedelta(days=amount))
                    elif "año" in time_type:
                        self.date_publication = dateToString(today - datetime.timedelta(days=amount*365))
                    else:
                        self.date_publication = "unknown"
                        print(f"Unknown {self.date_publication}")
                else:
                    self.date_publication = "unknown"

            self.state = self.extract_first(scrapper.xp("/html[1]/body[1]/div[2]/main[1]/div[1]/div[1]/article[1]/div[1]/div[3]/div[2]/div[1]")).replace("Etapa:","").strip()
            self.deliver = self.extract_first(scrapper.xp("/html[1]/body[1]/div[2]/main[1]/div[1]/div[1]/article[1]/div[1]/div[3]/div[2]/div[2]")).replace("Entrega:","").strip()
            
            price_items_split = self.extract_first(scrapper.driver.find_elements(By.XPATH, "//div[@class='price-items']")).split("\n")
            self.coin, self.price = re.match("([A-z]+) ([\d\.]+)", price_items_split[0]).groups()
            self.price = float(self.price.replace(".",""))
            self.discount = 0 if len(price_items_split) == 1 else int(price_items_split[1].replace("%",""))
            
            self.expenses = self.extract_first(scrapper.xp("//div[contains(@class,'block-expensas')]"))

            

            self.description_sup = self.extract_first(scrapper.driver.find_elements(By.XPATH, "//h2[@class='title-type-sup']")).replace("\n","|")
            self.description = self.extract_first(scrapper.driver.find_elements(By.XPATH, "//div[@id='longDescription']")).replace("\n","|")
            
            web_elements_imgs = scrapper.driver.find_elements(By.XPATH, "//div[@id='react-gallery']//img")
            self.pictures = ""
            for i,element in enumerate(web_elements_imgs):
                self.pictures += element.get_attribute("src")
                if i < len(web_elements_imgs)-1:
                    self.pictures += "|"
            
            self.advertiser_name = self.extract_first(scrapper.driver.find_elements(By.XPATH, "//section[@id='reactPublisherData']/div/div")).replace("\n","|")
            self.advertiser_codes = self.extract_multiple(scrapper.driver.find_elements(By.XPATH, "//section[@id='reactPublisherData']/div/ul/li")).replace("\n","|")

            features = scrapper.xp("//div[@class='nf-container']//div[@class='label']")
            for feature in features:
                text = feature.text.lower()
                
                if "total" in text:
                    self.feature_stotal = text
                elif "unid" in text:
                    self.feature_uni = text
                elif "cubie" in text:
                    self.feature_scover = text
                elif "baño" in text:
                    self.feature_bath = text
                elif "estac" in text:
                    self.feature_parking = text
                elif "habit" in text:
                    self.feature_rooms = text
                else:
                    print(f"unknown {text}")


            general_features = scrapper.xp("//div[@id='reactGeneralFeatures']//section")
            self.general_features_xml = "<general_features_xml>"
            for feature in general_features:
                type_title = feature.find_element(By.XPATH, "./div").text.strip()
                content = feature.find_element(By.XPATH, "./ul").text.strip().replace("\n","|")

                element = f"<general_feature><title>{type_title}</title><content>{content}</content></general_feature>"
                self.general_features_xml += element

            self.general_features_xml += "</general_features_xml>"
        else:
            print(self.type_real_state, url)            
            pass



        return self.getAttributesAsDict()

    def getAttributesAsDict(self):
        return self.__dict__


    def save(self,filename):
        with open(filename,"w") as file:
            json.dump(self.getAttributesAsDict(), file, ensure_ascii=False, indent=4)
    
    def set_attributes(self, dict_attributes:dict):
        self.attributes = []
        for key,value in dict_attributes.items():
            setattr(self, key, value)
            self.attributes.append(key)




In [141]:
from tqdm.notebook import tqdm
df = pd.DataFrame(columns=["type_real_state","url","title","location_string","location_latitude","location_longitude","date_publication","state","deliver","coin","views","price","expenses","discount","description_sup","description","pictures","advertiser_name","advertiser_codes","feature_rooms","feature_unities","feature_stotal","feature_scover","feature_bath","feature_parking","feature_toilete","feature_antique","general_features_xml","location_tree"])
index_df = 0
s = f"{datetime.datetime.now()}"

counter = 429
counter_end = 1000

try:
    debug_step = int(counter_end/10)
    for i in tqdm(range(counter,counter_end)):
        counter = i
        url = urls[counter]
        scrapper.driver.get(url)
        item = ItemRealState()
        item.parse_attributes(scrapper)
        df.at[index_df, :] = item.get_serie()
        index_df += 1
        counter += 1
        # a.save(f"temp/{counter}.json")
except Exception as e:
    print("Error: ", e, counter, url, counter_end)
finally:
    print(counter,counter_end)
    df.to_csv(f"shards/{s}.csv",index=False)


1000 1000


In [59]:

            
"""
self.cache["urls"] = urls_items
self.cache["index_seen"] = -1
self.cache["urls_error"] = []
self.cache["df"] = df

"""
import datetime
import json
##
restarting = False
cache = dict()
stop = False
##



try:
    
    for counter in range(0,10):
        url = urls[counter]
        counter += 1
        scrapper.driver.get(url)

        
        
finally:
    print("counter", counter)





oficina comercial https://www.plusvalia.com/propiedades/-0-9-8-2-9-5-3-1-9-3-vendo-oficina-torre-boreal-61870822.html

{'casa': 3, 'departamento': 1, 'oficina comercial': 1}
counter 4
